In [ ]:
# General imports
import pandas as pd
import numpy as np
import os
import sys
import re
import random
import json

import nltk
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [ ]:
# PARAMS
LANG = 'en'
EMB = 'bert'
TAG_TYPE = 'doc2vec'

In [ ]:
# Import utils class
sys.path.insert(0,'../')
from utils import Utils

# Instanciate utils class
utils = Utils(r'D:\Cesard\Documents\NLP', num_workers=10)

In [ ]:
# Load manual tags
with open(f'tags/manual/reddit_{lang}.json', 'r+') as file_str:
    reddit_manual_tags = json.load(file_str)
with open(f'tags/manual/tweets_{lang}.json', 'r+') as file_str:
    twitter_manual_tags = json.load(file_str)

# Load keyword tags
with open(f'tags/keywords/reddit_{lang}_words.json', 'r+') as file_str:
    reddit_kw_tags = json.load(file_str)
with open(f'tags/keywords/tweets_{lang}_words.json', 'r+') as file_str:
    twitter_kw_tags = json.load(file_str)

# Load Tagged data
print('Starting to load manual tagged data...')
manual_reddit_data, manual_reddit_file_names = utils.tagged_data_loader(list(reddit_manual_tags.keys()), 'reddit', lang)
manual_twitter_data, manual_twitter_file_names = utils.tagged_data_loader(list(twitter_manual_tags.keys()), 'tweets', lang)
print(f'Loaded {len(tagged_twitter_data)} tagged Tweets {len(tagged_reddit_data)} and tagged Reddit docs')
print('')

# Load Keyword tagged data
print('Starting to load keyword tagged data...')
reddit_data, reddit_file_names = utils.tagged_data_loader(list(reddit_kw_tags.keys()), 'reddit', lang)
twitter_data, twitter_file_names = utils.tagged_data_loader(list(twitter_kw_tags.keys()), 'tweets', lang)
print(f'Loaded {len(twitter_data)} tagged Tweets {len(reddit_data)} and tagged Reddit docs')

In [ ]:
def get_embedings()